##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Simulaciones de alto rendimiento con TFF

En este tutorial se describe cómo configurar simulaciones de alto rendimiento con TFF en distintos escenarios comunes.

TODO: b/134543154 - Complete el contenido, algunos de los temas que se deben cubrir aquí:

- uso de GPU en una configuración de una sola máquina,
- configuración de varias máquinas en GCP/GKE, con y sin TPU,
- interconexión de backends tipo MapReduce,
- limitaciones actuales y cuándo/cómo se flexibilizarán.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/simulations"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/simulations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/simulations.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/simulations.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

## Antes de empezar

Primero, asegúrese de que su notebook esté conectado a un backend que tenga compilados los componentes relevantes (incluidas las dependencias de gRPC para escenarios de múltiples máquinas).

Ahora, comencemos cargando el ejemplo MNIST desde el sitio web de TFF y declarando la función de Python que ejecutará un pequeño bucle experimental en un grupo de 10 clientes.

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

/bin/sh: pip: command not found


In [ ]:
import collections
import time

import tensorflow as tf

import tensorflow_federated as tff

source, _ = tff.simulation.datasets.emnist.load_data()


def map_fn(example):
  return collections.OrderedDict(
      x=tf.reshape(example['pixels'], [-1, 784]), y=example['label'])


def client_data(n):
  ds = source.create_tf_dataset_for_client(source.client_ids[n])
  return ds.repeat(10).shuffle(500).batch(20).map(map_fn)


train_data = [client_data(n) for n in range(10)]
element_spec = train_data[0].element_spec


def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(units=10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])
  return tff.learning.models.from_keras_model(
      model,
      input_spec=element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(0.02))


def evaluate(num_rounds=10):
  state = trainer.initialize()
  for _ in range(num_rounds):
    t1 = time.time()
    result = trainer.next(state, train_data)
    state = result.state
    train_metrics = result.metrics['client_work']['train']
    t2 = time.time()
    print('train metrics {m}, round time {t:.2f} seconds'.format(
        m=train_metrics, t=t2 - t1))

## Simulaciones de una sola máquina

Ahora activado de forma predeterminada.

In [ ]:
evaluate()

train metrics OrderedDict([('sparse_categorical_accuracy', 0.15329218), ('loss', 2.918891), ('num_examples', 9720), ('num_batches', 490)]), round time 4.64 seconds
train metrics OrderedDict([('sparse_categorical_accuracy', 0.18004115), ('loss', 2.7677088), ('num_examples', 9720), ('num_batches', 490)]), round time 2.37 seconds
train metrics OrderedDict([('sparse_categorical_accuracy', 0.21841563), ('loss', 2.511075), ('num_examples', 9720), ('num_batches', 490)]), round time 2.30 seconds
train metrics OrderedDict([('sparse_categorical_accuracy', 0.27160493), ('loss', 2.340346), ('num_examples', 9720), ('num_batches', 490)]), round time 2.25 seconds
train metrics OrderedDict([('sparse_categorical_accuracy', 0.34115225), ('loss', 2.0537064), ('num_examples', 9720), ('num_batches', 490)]), round time 2.27 seconds
train metrics OrderedDict([('sparse_categorical_accuracy', 0.3745885), ('loss', 1.9158486), ('num_examples', 9720), ('num_batches', 490)]), round time 2.21 seconds
train metrics 

## Simulaciones de múltiples máquinas en GCP/GKE, GPU, TPU y más...

Muy pronto.